In [1]:
from pathlib import Path
import pandas as pd
import json
import mavehgvs
import os
import numpy as np
from tqdm import tqdm
from Bio.PDB.Polypeptide import one_to_index,index_to_three
tqdm.pandas()
dataset_path = Path("/data/dzeiberg/mave_calibration/data/SCN5A/")
assert dataset_path.exists()

AUTHOR_TRANSCRIPT = 'ENST00000333535'

In [2]:
data = pd.read_excel(dataset_path / "raw" / "supp_file_s1.xlsx")
data = data.assign(author_transcript = AUTHOR_TRANSCRIPT)

/home/dzeiberg/.cache/pypoetry/virtualenvs/mave-calibration-xCjwkSk6-py3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
data

,mutation,mutation_type,aa_num,wt_allele,mut_allele,dms,dms_se,dms_95ci,dms_95ci_lower,dms_95ci_upper,class,author_transcript
0,L1621X,nonsense,1621,L,X,-11.5,3.4,6.6,-18.1,-4.9,LOF,ENST00000333535
1,F1622X,nonsense,1622,F,X,14.2,1.5,2.9,11.3,17.1,LOF,ENST00000333535
2,R1623X,nonsense,1623,R,X,-0.8,2.3,4.5,-5.2,3.7,LOF,ENST00000333535
3,V1624X,nonsense,1624,V,X,25.5,3.7,7.2,18.3,32.6,LOF,ENST00000333535
4,I1625X,nonsense,1625,I,X,-6.3,2.7,5.2,-11.5,-1.1,LOF,ENST00000333535
...,...,...,...,...,...,...,...,...,...,...,...,...
247,R1632S,nonsynonymous,1632,R,S,117.6,8.5,16.6,101.0,134.3,possiblyWT,ENST00000333535
248,R1632T,nonsynonymous,1632,R,T,115.2,6.1,12.0,103.2,127.3,possiblyWT,ENST00000333535
249,R1632V,nonsynonymous,1632,R,V,88.1,10.2,20.0,68.1,108.1,possiblyWT,ENST00000333535
250,R1632W,nonsynonymous,1632,R,W,44.6,12.0,23.5,21.0,68.1,LOF,ENST00000333535


In [4]:
data = data.rename(columns={'dms' : 'score'})

In [5]:
one_to_three = lambda x: index_to_three(one_to_index(x)).title() if x != "X" else "Ter"
def make_hgvs(r):
    if r.wt_allele == r.mut_allele:
        return f"p.{one_to_three(r.wt_allele)}{r.aa_num}="
    return f"p.{one_to_three(r.wt_allele)}{r.aa_num}{one_to_three(r.mut_allele)}"

In [6]:
data = data.assign(hgvs_pro = data.progress_apply(make_hgvs,axis=1))

100%|██████████| 252/252 [00:00<00:00, 41929.73it/s]


In [8]:
data = data[data.score.notnull()]

In [9]:
data

,mutation,mutation_type,aa_num,wt_allele,mut_allele,score,dms_se,dms_95ci,dms_95ci_lower,dms_95ci_upper,class,author_transcript,hgvs_pro
0,L1621X,nonsense,1621,L,X,-11.5,3.4,6.6,-18.1,-4.9,LOF,ENST00000333535,p.Leu1621Ter
1,F1622X,nonsense,1622,F,X,14.2,1.5,2.9,11.3,17.1,LOF,ENST00000333535,p.Phe1622Ter
2,R1623X,nonsense,1623,R,X,-0.8,2.3,4.5,-5.2,3.7,LOF,ENST00000333535,p.Arg1623Ter
3,V1624X,nonsense,1624,V,X,25.5,3.7,7.2,18.3,32.6,LOF,ENST00000333535,p.Val1624Ter
4,I1625X,nonsense,1625,I,X,-6.3,2.7,5.2,-11.5,-1.1,LOF,ENST00000333535,p.Ile1625Ter
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,R1632Q,nonsynonymous,1632,R,Q,61.0,11.5,22.6,38.4,83.5,possiblyLOF,ENST00000333535,p.Arg1632Gln
247,R1632S,nonsynonymous,1632,R,S,117.6,8.5,16.6,101.0,134.3,possiblyWT,ENST00000333535,p.Arg1632Ser
248,R1632T,nonsynonymous,1632,R,T,115.2,6.1,12.0,103.2,127.3,possiblyWT,ENST00000333535,p.Arg1632Thr
249,R1632V,nonsynonymous,1632,R,V,88.1,10.2,20.0,68.1,108.1,possiblyWT,ENST00000333535,p.Arg1632Val


In [10]:
data.to_csv(dataset_path / "scoreset.csv",index=False)

In [18]:
metadata = dict(uniprot_acc="Q14524")
with open(dataset_path / 'metadata.json','w') as f:
    json.dump(metadata,f)